# Employee Retention Analysis

- Nama: Nantha Seutia
- Email: ombakrinai@gmail.com
- Id Dicoding: ombakrinai@gmail.com

## Data Gathering

### Menyiapkan library yang dibutuhkan

In [ ]:
# Import Library
import csv
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from scipy.stats import pearsonr, spearmanr


### Menyiapkan data yang akan digunakan

In [ ]:
# Data Gathering
# Unduh file hanya sekali dan simpan lokal
url = "https://raw.githubusercontent.com/dicodingacademy/dicoding_dataset/main/employee/employee_data.csv"
local_path = "employee_data.csv"

# Simpan lokal
import requests
with open(local_path, 'wb') as f:
    f.write(requests.get(url).content)

# Baca dari file lokal (lebih cepat)
df = pd.read_csv(local_path, encoding="utf-8")
df.head()

## Data Understanding

In [ ]:
# Menampilkan ringkasan umum tentang dataset

# 1. Ukuran Dataset
print(f"📊 Dataset memiliki {df.shape[0]} baris dan {df.shape[1]} kolom.\n")

# 2. Nama-nama Kolom
print("🧾 Kolom-kolom dalam dataset:")
print(", ".join(df.columns), "\n")

# 3. Jumlah Data Duplikat
duplicates = df.duplicated().sum()
print(f"🔁 Jumlah baris duplikat: {duplicates}\n")

# 4. Kolom dengan Missing Values
missing = df.isnull().sum()[df.isnull().sum() > 0]
if not missing.empty:
    print("⚠️ Kolom yang memiliki nilai hilang (missing values):")
    for col, val in missing.items():
        print(f"- {col}: {val}")
else:
    print("✅ Tidak ada missing values dalam dataset.")


In [ ]:
df.info()

* Kolom Attrition hanya memiliki 1058 non-null dari 1470 → hilang sebanyak 412 entri.
* Perlu pemeriksaan apakah data hilang berkaitan dengan korelasi tertentu atau acak.
* Satu kolom numerik float: Attrition (seharusnya biner: 0 atau 1) bisa dikonversi ke int.
* 8 kolom kategorikal: object → perlu encoding pada tahap Data Preparation.

In [ ]:
df.describe()

In [ ]:
# Mengecek Distribusi Target
sns.countplot(data=df, x="Attrition")
plt.title("Distribusi Attrition (0 = Bertahan, 1 = Keluar)")
plt.show()

df["Attrition"].value_counts(dropna=False)

* Dataset tidak seimbang. Perlu penanganan khusus di tahap modeling: misalnya dengan resampling, class weighting, dsb.
* 28% missing pada target harus ditangani sebelum modeling.

In [ ]:
# Distribusi Target & Fitur Numerik
df.hist(figsize=(20, 15), bins=30)
plt.suptitle("Distribusi Fitur Numerik")
plt.show()

* Distribusi merata: Age, HourlyRate, MonthlyRate, DailyRate, PercentSalaryHike
* Distribusi skewed: DistanceFromHome, YearsAtCompany, YearsSinceLastPromotion, YearsWithCurrManager, YearsInCurrentRole
* Distribusi skewed tinggi: MonthlyIncome, TotalWorkingYears




In [ ]:
# Heatmap Korelasi:
# * Melihat kekuatan hubungan antar fitur
# * Menilai fitur apa yang berpengaruh terhadap target (Attrition)

# Continuous numeric features (exclude ordinal + Attrition)
numeric_features = ['Age', 'DailyRate', 'DistanceFromHome', 'HourlyRate', 'MonthlyIncome', 'MonthlyRate',
                      'NumCompaniesWorked', 'PercentSalaryHike', 'TotalWorkingYears', 'TrainingTimesLastYear',
                      'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager']

# Pearson correlation for continuous numeric
pearson_corr = df[numeric_features + ["Attrition"]].corr(method="pearson")
plt.figure(figsize=(12, 10))
sns.heatmap(pearson_corr, annot=True, fmt=".2f", cmap="coolwarm", square=True, linewidths=.5)
plt.title("Pearson Correlation: Continuous Numeric Features & Attrition")
plt.show()

# Print sorted correlation with Attrition
print("Pearson correlations with Attrition (continuous features):")
print(pearson_corr["Attrition"].drop("Attrition").sort_values(ascending=False))

In [ ]:
# Define ordinal feature groups
ordinal_features = ['Education', 'EnvironmentSatisfaction', 'JobInvolvement', 'JobLevel',
                    'JobSatisfaction', 'RelationshipSatisfaction', 'WorkLifeBalance']

# Spearman correlation for ordinal features
spearman_corr = df[ordinal_features + ["Attrition"]].corr(method="spearman")
plt.figure(figsize=(8, 6))
sns.heatmap(spearman_corr, annot=True, fmt=".2f", cmap="viridis", square=True, linewidths=.5)
plt.title("Spearman Correlation: Ordinal Features & Attrition")
plt.show()

print("\nSpearman correlations with Attrition (ordinal features):")
print(spearman_corr["Attrition"].drop("Attrition").sort_values(ascending=False))

**Catatan**: Korelasi dihitung menggunakan **Pearson** untuk fitur numerik kontinu dan **Spearman** untuk fitur ordinal.

### 1. Korelasi Positif (semakin besar, semakin cenderung keluar)
- **DistanceFromHome**: +0.078  
- **NumCompaniesWorked**: +0.037  
- **MonthlyRate**: +0.023  

> Semua korelasi tergolong **sangat lemah** (< |0.10|).

---

### 2. Korelasi Negatif (semakin besar, semakin cenderung bertahan)  
Fokus pada nilai **≤ –0.12**:

| Fitur                       | Korelasi  | Tipe      |
|-----------------------------|----------:|-----------|
| **JobLevel**                | –0.1918   | Ordinal   |
| **TotalWorkingYears**       | –0.1771   | Numeric   |
| **Age**                     | –0.1721   | Numeric   |
| **StockOptionLevel**        | –0.1642   | Numeric   |
| **MonthlyIncome**           | –0.1636   | Numeric   |
| **YearsInCurrentRole**      | –0.1588   | Numeric   |
| **YearsWithCurrManager**    | –0.1560   | Numeric   |
| **JobInvolvement**          | –0.1366   | Ordinal   |
| **YearsAtCompany**          | –0.1351   | Numeric   |
| **EnvironmentSatisfaction** | –0.1267   | Ordinal   |

> Fitur-fitur ini memiliki hubungan negatif terkuat dengan `Attrition`—nilai yang lebih tinggi menunjukkan peluang churn yang lebih rendah.

---

## Fitur yang Direkomendasikan untuk Dibuang

1. **Tidak Variatif (Konstan)**  
   - `EmployeeCount`  
   - `StandardHours`  
   - `Over18`

2. **Korelasi Sangat Lemah dengan Attrition**  
   - `MonthlyRate`  
   - `DailyRate`  
   - `HourlyRate`


## Data Preparation / Preprocessing

1. Tangani missing values (Attrition) & konversi ke integer
2. Drop kolom konstan/identifier
3. Encoding fitur kategorikal/ordinal

In [ ]:
# 1. Ubah fitur Attrition ke int64
df = df.dropna(subset=["Attrition"]).copy()
df["Attrition"] = df["Attrition"].astype(int)

# 2. Drop kolom konstan dan korelasi lemah
cols_to_drop = [
    "EmployeeId", "EmployeeCount", "StandardHours", "Over18",
    "MonthlyRate", "DailyRate", "HourlyRate"
]
df = df.drop(columns=cols_to_drop)

In [ ]:
# 3. Periksa fitur apa yang menggunakan Ordinal Encoding atau One-Hot Encoding
# Ordinal Encoding dipakai untuk urutan (mis. OverTime, Gender dalam konteks biner).
# One-Hot Encoding untuk fitur nominal banyak level seperti MaritalStatus.
df.select_dtypes(include=['object']).nunique()

In [ ]:
# 3.1 Ordinal Encoding for binary categorical features
ordinal_features = ["Gender", "OverTime"]
ord_enc = OrdinalEncoder()
df[ordinal_features] = ord_enc.fit_transform(df[ordinal_features])

# 3.2 One-Hot Encoding for all nominal features including BusinessTravel
nominal_features = ["BusinessTravel", "Department", "EducationField", "JobRole", "MaritalStatus"]
ohe = OneHotEncoder(drop='first', sparse_output=False, dtype=int)
X_ohe = ohe.fit_transform(df[nominal_features])
ohe_cols = ohe.get_feature_names_out(nominal_features)
df_ohe = pd.DataFrame(X_ohe, columns=ohe_cols, index=df.index)

# 3.3 Concatenate and drop original nominal columns
df = pd.concat([df.drop(columns=nominal_features), df_ohe], axis=1)

In [ ]:
# 3.4 Display resulting DataFrame info
print("Final DataFrame shape:", df.shape)
df.info()

In [ ]:
df.head()

## Modeling

## Evaluation